In [ ]:
import os
import requests
import re
from bs4 import BeautifulSoup as BS
from bs4 import SoupStrainer
from selenium import webdriver
from contextlib import contextmanager
from selenium.common.exceptions import NoSuchElementException
from time import sleep
import asyncio

In [ ]:
!rm -rf download

создадим папку с названием `download`

In [ ]:
PATH_TO_DOWNLOAD = 'download'
if not os.path.exists(PATH_TO_DOWNLOAD):
    os.mkdir(PATH_TO_DOWNLOAD)

In [ ]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

prefs = {
    "download.default_directory": f'{os.getcwd() + "/" + PATH_TO_DOWNLOAD + "/"}',
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
}

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome('chromedriver', options=chrome_options)

SITE = "http://www.infoeco.ru/"

# переходим на страницу 2017 года
def get_front():
    driver.get(f"{SITE}index.php?id=3122")
    html = driver.page_source
    soup = BS(html, 'html.parser')
    divs = soup.find_all("ul", class_="nav")[1].find_all("li")
    pat = re.compile(r"\d{4}")
    needed_hrefs = [d.a['href'] for d in divs if pat.search(d.a['title'])]
    driver.quit()
    if not needed_hrefs:
        get_front()
    return needed_hrefs
    
needed_hrefs = get_front()
needed_hrefs

In [ ]:
def download_year_content(idx=1, href="/index.php?id=3122"):
    '''
    input: 
        idx = year - 2016
        href = main link for 2017 eco data    
    '''
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    driver.get(f"{SITE}{href}")
    html = driver.page_source
    soup = BS(html, 'html.parser')
    divs = soup.find_all("ul")[1].find_all("li")
    div = divs[idx] # здесь хранятся данные по годам
    
    for i in div.find_all("a")[1:13]: # первый тег - год, нужны месяцы январь - декабрь
        driver.get(f"{SITE}{i['href']}")
        href = i['href']
    
        try:
            sleep(1)
            driver.find_element_by_xpath(f"//a[@href='{href}']").click()
            sleep(1)
        except NoSuchElementException:
            print(href, 'rip')
            continue

        html = driver.page_source
        only_section = SoupStrainer("section")
        soup = BS(html, 'html.parser', parse_only=only_section)
        snd_page = soup.find("div", class_="pager").a['href'] # вторая страница
        
        def loader(second_page=False):
            if second_page:
                try:
                    driver.get(f"{SITE}{snd_page}")
                    sleep(1)
                except NoSuchElementException:
                    print("there's no second page")
                    return
            html = driver.page_source
            soup = BS(html, 'html.parser', parse_only=only_section)
            divs = soup.find_all('div')
            for div in divs:
        
                href = div.a['href']
                try:
                    session = requests.Session()
                    if session.get(f"{SITE}{href}", headers=headers).status_code != 200:
                        print(f"{href} is broken! {date}")
                        continue
                    sleep(1)
                    driver.find_element_by_xpath(f"//a[@href='{href}']").click()
                    sleep(1)
                except:
                    print(f"{href} is stuck here ({date})")
                    continue

        loader()
        loader(second_page=True)

##### Описание
следующие 4 ячейки будут скачивать файлы для 2017 - н.в.

в 2016 у них очень странная структра (в одном файле записаны данные сразу за несколько дней, поэтому я решил не включать его в парсинг

первый аргумент - это индекс, он отвечает за год, e.g. `1 == 2017`, `2 == 2018` ...

в принте должны выдаваться сообщения о том, за какие числа браузер не смог скачать файлы
если ячейка закончила работу очень быстро - запуск еще раз

P.S. файлы скачиваются довольно долго (полчаса - час)

In [ ]:
download_year_content(1, needed_hrefs[1])

In [ ]:
download_year_content(2, needed_hrefs[2])

In [ ]:
download_year_content(3, needed_hrefs[3])

In [ ]:
download_year_content(4, needed_hrefs[4])

заплатка для названия файлов

формат: `ddmmyyyy.docx`

In [ ]:
date_names = [] #  новые соотв. названия файлов

@contextmanager
def rename_docx_files(dest, new_names):
    try:
        cwd = os.getcwd()
        os.chdir(dest)
        files = sorted(os.listdir(), key=os.path.getmtime)
        start = re.compile(r'^201\d')
        end = re.compile(r'201\d$')
        for i in files:
            try:
                x = re.search('\d[^a-zA-Z]*\d', i).group(0)
                if '-' in x or '_' in x or '.' in x:
                    x = re.sub(r'[._-]', '', x)
                if '00' in x and len(x) > 8:
                    x = re.sub('00', '0', x)
                if re.search(start, x) and not re.search(end, x):
                    'reversed date'
                    year = x[:4]
                    month = x[4:6]
                    day = x[6:8]
                    x = day + month + year
                if len(x) < 8:
                    x = x[:-2] + '20' + x[-2:]
                if x == "0328032018":
                    x = "03282018"
                if len(x) > 8:
                    x = x[:8]
                date_names.append(x)
        except:
            print('WTF', i) #  не должен триггериться (имхо)
        
        # переименовываем
        for i in range(len(files)):
            try:
                os.rename(files[i], new_names[i] + ".docx")                
            except FileNotFoundError:
                print(f"no such file {files[i]}")
        yield
    finally:
        os.chdir(cwd)

In [ ]:
with rename_docx_files(PATH_TO_DOWNLOAD, date_names):
    pass

In [ ]:
!ls download | wc -l